In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from pprint import pprint
import operator



def encode_sen(sen,corpus):
    """
    input: sentence and corpus
    output :  bag of words vector of sentence
    """
    cv = CountVectorizer()
    cv = cv.fit(corpus)
    vec = cv.transform([sen]).toarray()
    return vec[0]

def cosin_distance(vector1, vector2):
    """
    input: two bag of words vectors of sentence
    output :  the similarity between the sentence

    """
    dot_product = 0.0
    normA = 0.0
    normB = 0.0
    for a, b in zip(vector1, vector2):
        dot_product += a * b
        normA += a ** 2
        normB += b ** 2
    if normA == 0.0 or normB == 0.0:
        return None
    else:
        return dot_product / ((normA * normB) ** 0.5)


def doc_list2str(doc_list):
    """
    transform the doc_list to str
    """
    docu_str = ""
    for wordlist in doc_list:
        docu_str += " ".join(wordlist)
    return docu_str


def MMR(doc_list,corpus):
    """
    input ：corpus and the docment you want to extract
    output :the abstract of the docment
    """
    Corpus = corpus
    docu = doc_list2str(doc_list)
    doc_vec = encode_sen(docu,Corpus)
    QDScore = {}
    ###calculate the  similarity of every sentence with the whole corpus
    for sen in doc_list:
        sen = " ".join(sen)

        sen_vec = encode_sen(sen,corpus)
        score = cosin_distance(sen_vec,doc_vec)
        QDScore[sen] = score


    n = 2
    alpha = 0.7
    Summary_set = []
    while n > 0:
        MMRScore = {}
        ### select the first sentence of abstract
        if Summary_set == []:
            selected = max(QDScore.items(), key=operator.itemgetter(1))[0]
            Summary_set.append(selected)

        Summary_set_str = " ".join(Summary_set)

        for sentence in QDScore.keys():
             #calculate MMR
            if sentence not in Summary_set:
                sum_vec = encode_sen(Summary_set_str, corpus)
                sentence_vec = encode_sen(sentence,corpus)
                MMRScore[sentence] = alpha * QDScore[sentence] - (1 - alpha) * cosin_distance(sentence_vec,sum_vec)
        selected = max(MMRScore.items(), key=operator.itemgetter(1))[0]
        Summary_set.append(selected)
        n -= 1
    # print(len(Summary_set))
    return  Summary_set


In [19]:
import jieba
# docment = "伴随着世界杯的落幕，俱乐部联赛筹备工作又成为主流，转会市场必然也会在世界杯的带动下风起云涌，不过对于在本届赛事上大放异彩的姆巴佩而言，\
# 大巴黎可以吃一颗定心丸，世界杯最佳新秀已经亲自表态：留在巴黎哪里也不去。在接受外媒采访时，姆巴佩表达了继续为巴黎效忠的决心。\
# “我会留在巴黎，和他们一起继续我的路途，我的职业生涯不过刚刚开始”，姆巴佩说道。事实上，在巴黎这座俱乐部，充满了内部的你争我夺。\
# 上赛季，卡瓦尼和内马尔因为点球事件引发轩然大波，而内马尔联合阿尔维斯给姆巴佩起“忍者神龟”的绰号也让法国金童十分不爽，\
# 为此，姆巴佩的母亲还站出来替儿子解围。而早在二月份，一场与图卢兹的比赛，内马尔也因为传球问题赛后和姆巴佩产生口角。\
# 由此可见，巴黎内部虽然大牌云集，但是气氛并不和睦。内马尔离开球队的心思早就由来已久，而姆巴佩也常常与其它俱乐部联系在一起，在躲避过欧足联财政公平法案之后，\
# 巴黎正在为全力留下二人而不遗余力。好在姆巴佩已经下定决心，这对巴黎高层而言，也算是任务完成了一半。本届世界杯上，姆巴佩星光熠熠，\
# 长江后浪推前浪，大有将C罗、梅西压在脚下的趋势，他两次追赶贝利，一次是在1/8决赛完成梅开二度，另一次是在世界杯决赛中完成锁定胜局的一球，\
# 成为不满20岁球员的第二人。另外他在本届赛事中打进了4粒入球，和格列兹曼并列全队第一。而对巴黎而言，他们成功的标准只有一条：欧冠。\
# 而留下姆巴佩，可以说在争夺冠军的路上有了仰仗，卡瓦尼在本届世界杯同样表现不错，内马尔虽然内心波澜，但是之前皇马官方已经辟谣没有追求巴西天王，\
# 三人留守再度重来，剩下的就是图赫尔的技术战术与更衣室的威望，对图赫尔而言，战术板固然重要，但是德尚已经为他提供了更加成功的范本，\
# 像团结法国队一样去团结巴黎圣日耳曼，或许这才是巴黎取胜的钥匙。"
docment = "0 引言 当前，随着汽车市场需求不断变化和汽车行业的飞速的发展,自动驾驶已经成为汽车驾驶领域的重要研究热点之一。然而,在自动驾驶应用场景下,目标检测技术是计算机视觉中的\
一个热点问题。车辆目标检测系统是自动安全驾驶检测技术的重要组成内容,为了确保上路安全,需要能够精准地检测出路面上所有目标。因此，高效精准的车辆目标检测技术对自动驾驶系统的发展起到至关重要的作用。\
目标元素检测方法即对一个图像信息中的具有可变元素数量的每个目标元素进行精准定位和精确分类，最终得到图像中多个目标的类别以及在图像中的位置。目前为止，基于计\
算机视觉的目标检测大致分为两大类：传统的目标检测算法和基于深度学习的目标检测算法。传统目标检测算法流程图如图 1 所示，首先将输入图片中的感兴趣区域进行选择，接下\
来在感兴趣的区域里进行特征提取和对提取的特征进行分类。但是传统目标检测方法的三部分检测过程繁琐，计算量大，不能满足实时监测的要求。随着深度学习的发展，基于深度学习的目标检测算法被提了出来。\
基于深度学习的目标检测算法主要通过 CNN 完成目标特征提取工作，最后通过分类回归层完成目标的分类和定位工作。分为两大类：Two-stage 目标检测算法和 One-stage 目标检\
测算法。Two-stage 目标检测算法会先生成一些候选区域(regionproposals)，这些区域有可能会包含一个待检测目标，紧接着再采取一些后续措施来区分每个候选区域里具体包含了那些\
目标。例如 R-CNN[1]、FastR-CNN[2]、FasterR-CNN[3]等；One-stage 目标检测算法主要通过一遍网络得出目标的位置和类别信息，例如 SSD[4]和 YOLO[5-7]等。所以，One-stage 目标检\
测算法整个过程只需要一步速度比较快。基于深度学习的目标检测算法在车辆目标检测领域吸引了许多研究人员。刘云霄等[8]人提出一种多任务卷积神经网络（Multi-Task Cascaded Convolutional Network，MTCNN），该\
神经网络用于检测在城市道路，并且在雨雾雪天气等复杂环境场景下的检测精度较好。王聪等[9]人随后提出了一种深度融合多层卷积残差特征神经网络(convolutional neural network,\
cnn)的多层高级车辆残差特征重用检测残差神经网络重用技术网络模型,他将不同的卷积残差神经网络特征分别计算进行高层深度识别融合,并且对各种深度融合后的所有高层车辆残差特征重用检测数据通道分别计算进行残差特征权重\
加权,提升了各种类型高层车辆残差特征重用检测的图像标准计算精度。REDMONJ 等[10]人随后再次提出一种通过高层残差特征神经网络技术，实现残差特征深度识别图像的多层车辆深度卷积高层残差特征神经网络\
(featurereuse-resnet, fr-resnet),以车辆高层深度残差特征神经网络技术模型为基础,通过多个低层、高层、多尺度残差特征图像输入，低层深度残差神经特征在多层车辆内部中实现特征识别和车辆高层残差特征深度识别图像的特征权重学习策略等相结合应用来有效率地实现\
对车辆高层深度残差特征神经网络的高层特征识别重用,在两个多层高尺度特征姿态图底层的两个车辆残差特征识别数据集上分别计算取得了较高的车辆特征重用识别图像分析计算精度。然而上述车辆目标检测都是基于非单阶段式的算法，它们的不足之处在于目标实时检测\
的速度较慢，对于车辆目标实时检测来说,检测的速度慢并不是有利于实时的检测车辆来对目标。为了满足实时车辆目标检测文章选择了 yolo 系列的 YOLOv5 算法，并以 YOLOv5 模型为基础通过 K-means 聚类重新获取数据集的边界框，更换原网络中的损失函数和非极大\
值抑制，对车辆目标检测效果良好，同时也改善了遮挡目标的检测。1 改进 YOLOv5 算法目标检测模型1.1 基于 K-means 聚类的先验框重选取在大多数场景下原始数据集中的数据并不能满足理想训练的需求，如果想要获取更多的数据量不仅仅会为其增加训练的成本也会带来更多的工作量。为了获得更好的数据集，最好\
的方式是对原始数据集进行适当的数据预处理。原 YOLOv5 算法方法是通过对应的 coco 数据集的设计聚类网络来设计生成 9 个 anchorboxes,每个不同尺寸的锚点特征结构图分别表示对应 3 个 anchor boxes。聚类网络训练阶段,需要通过计算真实框与哪个 anchor boxes 的间距 IoU 最大,标记确定该真实\
锚框与其对应的锚点置信度差值为 1。Anchor boxes 用来预测 bounding box 的，YOLOv5 算法中的 anchorboxes 是由 COCO 数据集得到的，在这些 anchor boxes 中，目标大小的差距是很大，如果是使用自己的数据集做目标检测，那么其中部分 anchor 的设计并不是最合理。文章采用的是\
KITTI数据集对改进后的YOLOv5算法进行测试，为了得到KITTI数据集最适合的anchorbox，通过采用 k-means 检测方法随机计算每个锚框的目标大小,即首先随机依次选取一个数据找到集中 k 个点将其作为集合聚类产物中心,然后针对每个数据集中的每个聚合样本类别计算\
其中找到集中 k 个点的聚合分类产物中心的目标距离并将其进行分类后放到目标距离最小的一个聚类产物中心所在相对应的样本类别中,接着再针对每个样本类别重新随机计算一个聚类产物中心,最后再次重复上述 2 个计算步骤,直到每个聚类产物中心的目标位置不再发生变化。通过计算 K-means 方法得到的每个锚框大小尺度有效地大大提升了它的 yolov5 算法\
测试性能。聚类结果如图 2 所示1.2 边界框检测 Loss 的改进目前基于 Anchor 预测机制的最小目标检测主要应用是通过测量最小化目标预测其中框物体坐标与扩大目标预测框物体坐标的高度均方差距离来不断改进目标预测框中物体的测量精度。在原始 YOLOv5 算法中 IoU_Lossv[11]损失函数采用的是 GIoU_Loss 损失函数。\
GIoU_Loss 也可以是一种损失函数的距离度量,可以直接满足基本损失距离函数的度量要求,同时由于 GIoU_Loss 还具有一种具有强的尺度不变性，表达式如下：但是 GIoU_Loss 存在着两框包含的时候，GIOU_Loss 会退化成 IOU_Loss 和 GIOU_Loss需要迭代很多次才能收敛，考虑到 GIou 的缺点，文章引入了 CIoU_Loss,表达式如下：\
分别表示预测框和目标框的中心点，两者之间距离采用欧式距离表示能同时包含预测框和目标框的最小包围框的斜距。CIoU_Loss 能够直接最小化预测框和真实框的中心点距离加速收敛，同时它还增加了可以检测真实框不同尺度的距离 loss,增加了长和宽的 loss,这样整个预测框就会更加的完全符合真实框。所以文章中 CIoU_Loss \
代替原来的 GIoU_Loss，效果会更好1.3 非极大值抑制 nms 改进在目标检测算法的最后处理阶段中，针对多目标框的筛选问题，通常需要非极大值Non-maximum suppression，NMS）算法去选择目标框，而在 NMS 算法里有一个步是需要计算当前 score 最大的框和其他框的 IoU 大小的。针对这一步，我们可以进行改进，改变 IoU\
的计算方式。传统的 NMS 主要用于对预测框的筛选，通过 IoU 索引来抑制冗余的预测框，而重叠部分会使筛选存在错误筛选信息。抑制冗余的预测框不仅要考虑重叠部分，也要考虑预测框和目标框的中心点距离。DIoU 则同时考虑到上述两个问题。因此，文章中将原网络的 NMS 更改为 DIoU_NMS，所以在针对重叠多个目标的网络检测中，\
DIOU_nms 的检测效果明显优于传统的 nms 算法，如图 3 所示，公式如下所示。"

sen_list = docment.strip().split("。")
sen_list.remove("")
doc_list = [jieba.lcut(i) for i in sen_list]
corpus = [" ".join(i) for i in doc_list]

In [20]:
res = ''
for s in MMR(doc_list,corpus):
    res += s.replace(' ', '')

In [21]:
res

'目前为止，基于计算机视觉的目标检测大致分为两大类：传统的目标检测算法和基于深度学习的目标检测算法王聪等[9]人随后提出了一种深度融合多层卷积残差特征神经网络(convolutionalneuralnetwork,cnn)的多层高级车辆残差特征重用检测残差神经网络重用技术网络模型,他将不同的卷积残差神经网络特征分别计算进行高层深度识别融合,并且对各种深度融合后的所有高层车辆残差特征重用检测数据通道分别计算进行残差特征权重加权,提升了各种类型高层车辆残差特征重用检测的图像标准计算精度为了满足实时车辆目标检测文章选择了yolo系列的YOLOv5算法，并以YOLOv5模型为基础通过K-means聚类重新获取数据集的边界框，更换原网络中的损失函数和非极大值抑制，对车辆目标检测效果良好，同时也改善了遮挡目标的检测'